In [20]:
!pip install flask-ngrok


In [25]:
# Cell 1 — install dependencies
!pip install -q streamlit yfinance pyngrok pandas


In [26]:
from flask import Flask
from flask_ngrok import run_with_ngrok

# Create Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app runs


In [27]:
# Cell 2 — write app.py
%%bash
cat > app.py <<'PY'
import streamlit as st
import yfinance as yf
import pandas as pd

st.set_page_config(page_title="Stock Consultant Agent", layout="centered")

BILL_PER_PORTFOLIO = 2
BILL_PER_ADVICE = 1

@st.cache_data(ttl=300)
def get_stock_info(symbol):
    """
    Fetch last 5 day close prices and compute % change between first and last close.
    For Indian NSE tickers we append .NS
    """
    try:
        ticker = symbol.upper().strip()
        # If user provided something that already contains a suffix like .NS, keep it
        if not (ticker.endswith(".NS") or ticker.endswith(".BO") or ticker.endswith(".NSX")):
            ticker = ticker + ".NS"
        stock = yf.Ticker(ticker)
        hist = stock.history(period="5d")
        if hist.empty or "Close" not in hist:
            return None
        price = hist["Close"].iloc[-1]
        change_pct = ((hist["Close"].iloc[-1] - hist["Close"].iloc[0]) / hist["Close"].iloc[0]) * 100
        sector = "Unknown"
        try:
            info = stock.info
            sector = info.get("sector", "Unknown")
        except Exception:
            sector = "Unknown"
        return {"Price": round(price, 2), "Change": round(change_pct, 2), "Sector": sector}
    except Exception:
        return None

def analyze_portfolio(portfolio):
    advice = []
    info_map = {}
    known_sectors = []
    for s in portfolio:
        info = get_stock_info(s)
        info_map[s] = info
        if info:
            sec = info.get("Sector", "Unknown")
            known_sectors.append(sec)
            if info["Change"] < -3:
                advice.append(f"You already hold {s}. Risk is high — consider reducing.")
            elif info["Change"] > 2:
                advice.append(f"{s} is performing well. Safe to hold for now.")
            else:
                advice.append(f"{s} is stable. You may continue holding.")
        else:
            advice.append(f"Data for {s} not available. Please check ticker.")
    # diversification rule
    non_unknown = [x for x in known_sectors if x != "Unknown"]
    if len(non_unknown) > 1 and len(set(non_unknown)) == 1:
        advice.append("Your portfolio is concentrated in one sector — consider diversification.")
    if not any((sec and sec.lower() == "it") for sec in non_unknown):
        advice.append("Your portfolio lacks IT stocks — consider adding an IT company such as Infosys or TCS.")
    return advice, info_map

# simple in-memory usage (resets on runtime restart)
if "usage" not in st.session_state:
    st.session_state.usage = {"portfolios": 0, "advices": 0, "bill": 0}

st.title("📊 Stock Consultant Agent (Colab demo)")
st.write("Enter tickers (comma separated). Example: `TCS, RELIANCE, INFY`")

portfolio_input = st.text_input("Enter portfolio:", "TCS, RELIANCE")

if st.button("Analyze Portfolio"):
    if not portfolio_input.strip():
        st.warning("Enter at least one ticker.")
    else:
        portfolio = [t.strip().upper() for t in portfolio_input.split(",") if t.strip()]
        advices, info_map = analyze_portfolio(portfolio)

        st.session_state.usage["portfolios"] += 1
        st.session_state.usage["advices"] += len(advices)
        st.session_state.usage["bill"] += BILL_PER_PORTFOLIO + BILL_PER_ADVICE * len(advices)

        st.subheader("📌 Advice (plain English)")
        for a in advices:
            st.write("- " + a)

        st.subheader("📑 Stock Data (latest)")
        rows = []
        for s in portfolio:
            info = info_map.get(s)
            if info:
                rows.append({"Ticker": s, "Price": info["Price"], "Change(5d%)": info["Change"], "Sector": info.get("Sector","Unknown")})
            else:
                rows.append({"Ticker": s, "Price": "N/A", "Change(5d%)": "N/A", "Sector": "N/A"})
        st.dataframe(pd.DataFrame(rows))

        st.subheader("📈 Usage & Billing (demo)")
        st.write(f"Portfolios analyzed: {st.session_state.usage['portfolios']}")
        st.write(f"Advices generated: {st.session_state.usage['advices']}")
        st.write(f"Total bill (mock): ₹{st.session_state.usage['bill']}")
PY


In [29]:
# Cell 3 — start streamlit in background
# NOTE: allow a few seconds for the server to start before creating tunnel
!nohup streamlit run app.py --server.port 8501 --server.headless true --server.enableCORS false > streamlit_out.log 2>&1 &
# show last lines of log after a short pause
!sleep 1
!tail -n 40 streamlit_out.log

2025-09-19 05:13:28.183 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            




In [31]:
# Cell 4 — connect ngrok (REPLACE the token string with your real ngrok v2 token)
from pyngrok import ngrok
import time

# kill previous tunnels (if any)
try:
    ngrok.kill()
except Exception:
    pass

# <-- REPLACE the string below with your real ngrok v2 authtoken (starts with "2") -->
NGROK_TOKEN = "32u8TJYIP9rzHir1jHLgdewvmSK_TGv9nuqUAHUZfRJiZDnc"

# register the token
ngrok.set_auth_token(NGROK_TOKEN)

# create tunnel to port 8501 (Streamlit)
tunnel = ngrok.connect(addr="8501", bind_tls=True)
time.sleep(1)
print("Public URL (open in browser):", tunnel.public_url)


Public URL (open in browser): https://0c079835b0b2.ngrok-free.app


In [32]:
# Cell 5 — show streamlit logs
!tail -n 200 streamlit_out.log


2025-09-19 05:13:28.183 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.45.74.159:8501



In [33]:
# Cell 6 — stop everything
from pyngrok import ngrok
ngrok.kill()
# kill streamlit process if present
!pkill -f streamlit || true
print("Stopped ngrok and streamlit (if running).")


^C
Stopped ngrok and streamlit (if running).


In [34]:
import yfinance as yf
print(yf.Ticker("RELIANCE.NS").history(period="5d").tail())


                                  Open         High          Low        Close  \
Date                                                                            
2025-09-15 00:00:00+05:30  1393.000000  1400.800049  1388.599976  1399.300049   
2025-09-16 00:00:00+05:30  1404.699951  1408.000000  1398.199951  1405.300049   
2025-09-17 00:00:00+05:30  1407.000000  1416.199951  1406.900024  1413.800049   
2025-09-18 00:00:00+05:30  1420.400024  1422.000000  1410.699951  1415.000000   
2025-09-19 00:00:00+05:30  1414.900024  1417.000000  1403.599976  1405.300049   

                            Volume  Dividends  Stock Splits  
Date                                                         
2025-09-15 00:00:00+05:30  5396697        0.0           0.0  
2025-09-16 00:00:00+05:30  8686961        0.0           0.0  
2025-09-17 00:00:00+05:30  7519417        0.0           0.0  
2025-09-18 00:00:00+05:30  9332642        0.0           0.0  
2025-09-19 00:00:00+05:30  2062958        0.0           0.0 

In [37]:
import pandas as pd

# Sample stock data (you can later connect to a live API)
data = {
    "Stock": ["TCS", "INFY", "HDFC", "RELIANCE", "WIPRO"],
    "Price": [3600, 1500, 2800, 2500, 550],
    "Sector": ["IT", "IT", "Finance", "Energy", "IT"]
}

stock_df = pd.DataFrame(data)
stock_df


,Stock,Price,Sector
0,TCS,3600,IT
1,INFY,1500,IT
2,HDFC,2800,Finance
3,RELIANCE,2500,Energy
4,WIPRO,550,IT


In [38]:
def analyze_portfolio(portfolio):
    advice = []

    for stock in portfolio:
        if stock not in stock_df["Stock"].values:
            advice.append(f"{stock}: Not found in database ❌")
            continue

        price = stock_df.loc[stock_df["Stock"] == stock, "Price"].values[0]

        if price > 3000:
            advice.append(f"{stock}: Price is high ({price}). Consider reducing ⚠️")
        elif price < 1000:
            advice.append(f"{stock}: Looks undervalued ({price}). Consider adding ✅")
        else:
            advice.append(f"{stock}: Fairly priced at {price}. Hold 🟢")

    # Diversification check
    sectors = stock_df[stock_df["Stock"].isin(portfolio)]["Sector"].unique()
    if len(sectors) < 2:
        advice.append("⚠️ Your portfolio lacks diversification. Try adding stocks from other sectors.")

    return advice


In [50]:
# @app.route("/analyze", methods=["GET"])
# def analyze_portfolio_route():
#     stocks = request.args.get("stocks", "")
#     portfolio = [s.strip().upper() for s in stocks.split(",") if s.strip()]

#     if not portfolio:
#         return "Please provide stocks in URL like: /analyze?stocks=TCS,INFY,HDFC"

#     result = analyze_portfolio(portfolio)
#     return "<br>".join(result)

In [59]:
!pip install gunicorn
!gunicorn -w 4 -b 0.0.0.0:5000 app:app


[2025-09-19 06:26:12 +0000] [22526] [INFO] Handling signal: int
[2025-09-19 06:26:12 +0000] [22530] [INFO] Worker exiting (pid: 22530)
[2025-09-19 06:26:12 +0000] [22529] [INFO] Worker exiting (pid: 22529)
[2025-09-19 06:26:12 +0000] [22527] [INFO] Worker exiting (pid: 22527)
[2025-09-19 06:26:12 +0000] [22528] [INFO] Worker exiting (pid: 22528)
[2025-09-19 06:26:12 +0000] [22526] [INFO] Shutting down: Master


In [58]:
!pip install pyngrok

from pyngrok import ngrok
from flask import Flask

app = Flask(__name__)

@app.route("/")
def home():
    return "Hello from Flask with pyngrok 🚀"

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

app.run(port=5000)


Public URL: NgrokTunnel: "https://e9e005231bcc.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [61]:
# Global counters
usage_stats = {
    "portfolios_analyzed": 0,
    "advices_generated": 0,
    "bill": 0
}

@app.route("/analyze", methods=["GET"])
def analyze_portfolio_route():
    stocks = request.args.get("stocks", "")
    portfolio = [s.strip().upper() for s in stocks.split(",") if s.strip()]

    if not portfolio:
        return "32u8TJYIP9rzHir1jHLgdewvmSK_TGv9nuqUAHUZfRJiZDnc"

    # Update usage stats
    usage_stats["portfolios_analyzed"] += 1
    usage_stats["bill"] += 10  # e.g. ₹10 per portfolio analyzed

    result = analyze_portfolio(portfolio)

    # Update advice count and billing
    usage_stats["advices_generated"] += len(result)
    usage_stats["bill"] += len(result) * 2  # e.g. ₹2 per advice

    return "<br>".join(result)

@app.route("/usage", methods=["GET"])
def show_usage():
    return f"""
    <h2>📊 Usage Stats</h2>
    Portfolios analyzed: {usage_stats['portfolios_analyzed']}<br>
    Advices generated: {usage_stats['advices_generated']}<br>
    Total Bill: ₹{usage_stats['bill']}
    """


AssertionError: View function mapping is overwriting an existing endpoint function: analyze_portfolio_route

### Set up ngrok tunnel for Flask app

This cell will set up an ngrok tunnel to port 5000, where the Flask app is running. Make sure the previous cell with the `gunicorn` command is still running.

In [64]:
from pyngrok import ngrok
import time

# kill previous tunnels (if any)
try:
    ngrok.kill()
except Exception:
    pass

# <-- REPLACE the string below with your real ngrok v2 authtoken (starts with "2") -->
NGROK_TOKEN = "32u8TJYIP9rzHir1jHLgdewvmSK_TGv9nuqUAHUZfRJiZDnc"

# register the token
ngrok.set_auth_token(NGROK_TOKEN)

# create tunnel to port 5000 (Flask)
tunnel = ngrok.connect(addr="5000", bind_tls=True)
time.sleep(1)
print("Public URL (open in browser):", tunnel.public_url)

Public URL (open in browser): https://4c369c968cec.ngrok-free.app


# Task
Explain the error in the selected code. If possible, fix the error and incorporate the changes into the existing code. Otherwise, try to diagnose the error.

## Modify app.py

### Subtask:
Update the `app.py` file to contain only the Flask application code and its routes (from the notebook cells `AHSl0okbcGQY` and `9oMZ9OaRdmEY`), removing the Streamlit specific code.


**Reasoning**:
The subtask is to update the `app.py` file with the Flask application code, removing the Streamlit code. This requires writing the complete Flask application including imports, data, functions, and routes into the `app.py` file.



In [56]:
%%writefile app.py
from flask import Flask, request
import pandas as pd

# Sample stock data (you can later connect to a live API)
data = {
    "Stock": ["TCS", "INFY", "HDFC", "RELIANCE", "WIPRO"],
    "Price": [3600, 1500, 2800, 2500, 550],
    "Sector": ["IT", "IT", "Finance", "Energy", "IT"]
}

stock_df = pd.DataFrame(data)

def analyze_portfolio(portfolio):
    advice = []

    for stock in portfolio:
        if stock not in stock_df["Stock"].values:
            advice.append(f"{stock}: Not found in database ❌")
            continue

        price = stock_df.loc[stock_df["Stock"] == stock, "Price"].values[0]

        if price > 3000:
            advice.append(f"{stock}: Price is high ({price}). Consider reducing ⚠️")
        elif price < 1000:
            advice.append(f"{stock}: Looks undervalued ({price}). Consider adding ✅")
        else:
            advice.append(f"{stock}: Fairly priced at {price}. Hold 🟢")

    # Diversification check
    sectors = stock_df[stock_df["Stock"].isin(portfolio)]["Sector"].unique()
    if len(sectors) < 2:
        advice.append("⚠️ Your portfolio lacks diversification. Try adding stocks from other sectors.")

    return advice

# Global counters
usage_stats = {
    "portfolios_analyzed": 0,
    "advices_generated": 0,
    "bill": 0
}

# Create Flask app
app = Flask(__name__)

@app.route("/")
def home():
    return "Hello from Flask! Use /analyze?stocks=TCS,INFY to analyze a portfolio."

@app.route("/analyze", methods=["GET"])
def analyze_portfolio_route():
    stocks = request.args.get("stocks", "")
    portfolio = [s.strip().upper() for s in stocks.split(",") if s.strip()]

    if not portfolio:
        return "Please provide stocks in URL like: /analyze?stocks=TCS,INFY,HDFC"

    # Update usage stats
    usage_stats["portfolios_analyzed"] += 1
    usage_stats["bill"] += 10  # e.g. ₹10 per portfolio analyzed

    result = analyze_portfolio(portfolio)

    # Update advice count and billing
    usage_stats["advices_generated"] += len(result)
    usage_stats["bill"] += len(result) * 2  # e.g. ₹2 per advice

    return "<br>".join(result)

@app.route("/usage", methods=["GET"])
def show_usage():
    return f"""
    <h2>📊 Usage Stats</h2>
    Portfolios analyzed: {usage_stats['portfolios_analyzed']}<br>
    Advices generated: {usage_stats['advices_generated']}<br>
    Total Bill: ₹{usage_stats['bill']}
    """

Overwriting app.py


## Run flask app with gunicorn

### Subtask:
Execute the gunicorn command in cell `9oMZ9OaRdmEY` to run the modified Flask application.


**Reasoning**:
Execute the gunicorn command to run the modified Flask application.



In [54]:
!gunicorn -w 4 -b 0.0.0.0:5000 app:app

[2025-09-19 05:52:29 +0000] [17452] [INFO] Starting gunicorn 23.0.0
[2025-09-19 05:52:29 +0000] [17452] [INFO] Listening at: http://0.0.0.0:5000 (17452)
[2025-09-19 05:52:29 +0000] [17452] [INFO] Using worker: sync
[2025-09-19 05:52:29 +0000] [17453] [INFO] Booting worker with pid: 17453
[2025-09-19 05:52:29 +0000] [17454] [INFO] Booting worker with pid: 17454
[2025-09-19 05:52:29 +0000] [17455] [INFO] Booting worker with pid: 17455
[2025-09-19 05:52:29 +0000] [17456] [INFO] Booting worker with pid: 17456
[2025-09-19 06:11:10 +0000] [17452] [INFO] Handling signal: int
[2025-09-19 06:11:10 +0000] [17455] [INFO] Worker exiting (pid: 17455)
[2025-09-19 06:11:10 +0000] [17454] [INFO] Worker exiting (pid: 17454)
[2025-09-19 06:11:10 +0000] [17456] [INFO] Worker exiting (pid: 17456)
[2025-09-19 06:11:10 +0000] [17453] [INFO] Worker exiting (pid: 17453)
[2025-09-19 06:11:11 +0000] [17452] [INFO] Shutting down: Master


In [66]:
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
